In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
from datetime import timedelta
import calendar
import seaborn as sns
import datetime

# 1. Get list of stocks new_in, old_out, old_in at the time of underlying changing
csi_index = pd.read_csv('D:\\work\\project 3 event study\\index_comp_SH000905.csv', encoding="utf-8")
csi_index_300 = pd.read_csv('D:\\work\\project 3 event study\\index_comp_SH000300.csv', encoding="utf-8")
csi_index_1000 = pd.read_csv('D:\\work\\project 3 event study\\index_comp_SH000852.csv', encoding="utf-8")
csi_rest = pd.read_csv('D:\\work\\project 3 event study\\index_comp_SH000985.csv', encoding="utf-8")

# ST_stocks = pd.read_csv('C:\\Users\\win\\Downloads\\STlist.csv', encoding="utf-8")
csi_index["Date"] = csi_index["Date"].apply(lambda x: str(x))
csi_index_300["Date"] = csi_index_300["Date"].apply(lambda x: str(x))
csi_index_1000["Date"] = csi_index_1000["Date"].apply(lambda x: str(x))
csi_rest["Date"] = csi_rest["Date"].apply(lambda x: str(x))
da_te = pd.to_datetime(csi_index["Date"])
sto_ck = pd.read_csv('F:\\all_stock_202004.csv', encoding = "GBK").iloc[:,1:]
sto_ck["returns"] = sto_ck.groupby("Symbol")['close'].apply(lambda x: x/x.shift(1)-1)
sto_ck["Date"] = sto_ck["Date"].apply(lambda x: str(x))
sto_ck["Date"] = pd.to_datetime(sto_ck["Date"])
xx = []
xx.append(csi_index.columns[csi_index.iloc[0, :] != 0])
new_in = pd.DataFrame()
old_in = pd.DataFrame()
for i in range(1, len(csi_index)):
    xx.append(csi_index.columns[csi_index.iloc[i, :] != 0])
    new_in[csi_index.iloc[i, 0]] = pd.Series(list(set(xx[i][1:]) - set(xx[i - 1][1:])))
    old_in[csi_index.iloc[i, 0]] = pd.Series(list(set(xx[i - 1][1:]) & set(xx[i][1:])))
new_in = new_in.dropna(axis=1, how="any")
old_in = old_in[new_in.columns]
old_in = old_in.dropna(axis=0, how="all")
hs300_list = csi_index_300.columns
zz500_list = csi_index.columns
zz1000_list = csi_index_1000.columns
zzall_list = csi_rest.columns


# change the value of total shares!!!!!!!!!!!!!!
st_list = sto_ck[(sto_ck["Name"].str.contains("ST") != False)  |
             (sto_ck["Name"].str.contains("退") != False)]["Symbol"].unique()
sto_ck = sto_ck[~sto_ck["Symbol"].isin(st_list)]
# sto_ck.loc[sto_ck["Symbol"] == "SZ000022", "Symbol"] = "SZ001872"
# sto_ck.loc[sto_ck["Symbol"] == "SZ000043", "Symbol"] = "SZ001914"

ts = pd.read_csv('F:\\total shares 202004.csv')
ts = ts.rename(columns={ts.columns[2]:"A股合计"})
ts = ts.rename(columns={ts.columns[3]:"B股合计"})
ts = ts.rename(columns={ts.columns[4]:"H股合计"})
ts["A股合计"] = ts["A股合计"].astype(str)
ts["B股合计"] = ts["B股合计"].astype(str)
ts["H股合计"] = ts["H股合计"].astype(str)
ts["A股合计"] = ts["A股合计"].apply(lambda x: float(x.replace(',', "")))
ts["B股合计"] = ts["B股合计"].apply(lambda x: float(x.replace(',', "")))
ts["H股合计"] = ts["H股合计"].apply(lambda x: float(x.replace(',', "")))
ts["证券代码"] = ts["证券代码"].str[-2:] + ts["证券代码"].str[:-3] 
sto_ck = pd.merge(sto_ck, ts, left_on="Symbol", right_on="证券代码", how="left")
display(sto_ck[sto_ck["A股合计"].isnull()]["Name"].unique())
sto_ck = sto_ck[~sto_ck["A股合计"].isnull()]

sto_ck["Totalshares"] = np.where((sto_ck["B股合计"] != 0)|(sto_ck["H股合计"] != 0), sto_ck["A股合计"], sto_ck["Totalshares"])
sto_ck["TotalValue"] = sto_ck["Totalshares"] * sto_ck["close"]

ir = pd.read_csv('F:\\2019年报净利润.csv', encoding = "GBK")
ir = ir.rename(columns={ir.columns[2]:"净利润"})
ir["净利润"] = np.where(ir["净利润"] == '——', 0, ir["净利润"])
ir["净利润"] = ir["净利润"].astype(str)
ir["净利润"] = ir["净利润"].apply(lambda x: float(x.replace(',', "")))
ir["证券代码"] = ir["证券代码"].str[-2:] + ir["证券代码"].str[:-3] 
sto_ck = pd.merge(sto_ck, ir, left_on="Symbol", right_on="证券代码", how="left")
display(sto_ck[sto_ck["净利润"].isnull()]["Name"].unique())



real_date = csi_index[csi_index["Date"] == csi_index["Date"].max()]["Date"].max()
# CSI500 old list
total_list = zz500_list[(csi_index[csi_index["Date"] == real_date] != 0).values[0]][1:]
# CSI500 new list
total_list1 = pd.concat([new_in.iloc[:, -3], old_in.iloc[:, -3]])
assert(len(total_list) == 500)
assert(len(total_list1) == 500)


# 第一部分：筛选沪深300样本指数成分股
# 1. 筛选样本空间

print(sto_ck["Date"].min())
print(sto_ck["Date"].max())
all_data = sto_ck[["Date", "Symbol","Name", "TotalValue", "MarketValue", "Totalshares", "Marketshares", "close", "amt", "ListDays", "净利润"]]

la_st = all_data.groupby("Symbol").last().reset_index()
old_list = hs300_list[(csi_index_300[csi_index_300["Date"] == real_date] != 0).values[0]][1:]
new_list = hs300_list[(csi_index_300[csi_index_300["Date"] == old_in.columns[-3]] != 0).values[0]][1:]
ol = zz1000_list[(csi_index_1000[csi_index_1000["Date"] == real_date] != 0).values[0]][1:]
nl = zz1000_list[(csi_index_1000[csi_index_1000["Date"] == old_in.columns[-3]] != 0).values[0]][1:]
ol1 = zzall_list[(csi_rest[csi_rest["Date"] == real_date] != 0).values[0]][1:]
nl1 = zzall_list[(csi_rest[csi_rest["Date"] == old_in.columns[-3]] != 0).values[0]][1:]
assert(len(old_list) == 300)
assert(len(new_list) == 300)
assert(len(ol) == 1000)
assert(len(nl) == 1000)
la_st["CSI500"] = np.nan
for i in range(len(total_list)):
    la_st.loc[la_st["Symbol"] == total_list[i], ["CSI500"]] = 1
la_st["HS300"] = np.nan
for i in range(len(old_list)):
    la_st.loc[la_st["Symbol"] == old_list[i], ["HS300"]] = 1
la_st["CSI1000"] = np.nan
for i in range(len(ol)):
    la_st.loc[la_st["Symbol"] == ol[i], ["CSI1000"]] = 1
# 非创业板股票上市时间超过一个季度，创业板股票上市时间超过三年
l1 = all_data.groupby("Symbol")["TotalValue"].mean().reset_index()
l1 = l1.rename(columns={'TotalValue': 'daily_tv'})
l2 = all_data.groupby("Symbol")["amt"].mean().reset_index()
l2 = l2.rename(columns={'amt': 'daily_amt'})
l3 = all_data.groupby("Symbol").size().reset_index(name='TradeDays')
la_st = pd.merge(la_st, l1, left_on="Symbol", right_on="Symbol")
la_st = pd.merge(la_st, l2, left_on="Symbol", right_on="Symbol")
la_st = pd.merge(la_st, l3, left_on="Symbol", right_on="Symbol")
la_st1 = la_st
cyb = la_st[(la_st["Symbol"].str[:3] == "SZ3") & (la_st["ListDays"] > 750)]["Symbol"]
fcyb = la_st[(la_st["Symbol"].str[:3] != "SZ3") & (la_st["ListDays"] > 60) &
             (la_st["Name"].str.contains("ST") == False) & (la_st["Symbol"].str[:5] != "SH688") &
             (la_st["Name"].str.contains("退") == False)]["Symbol"]  # 2691只股票，不存在上市小于一季度日均总市值前30的情况
la_st = la_st[la_st["Symbol"].isin(cyb) | la_st["Symbol"].isin(fcyb)]

#至数据考察截止日已连续停止交易超过25个交易日且仍未恢复交易的，这个之前没用上
# t_s = t_s[t_s["Symbol"].isin(la_st["Symbol"])]
# li_st = t_s.groupby("Symbol")["trade_stats"].rolling(25).sum().reset_index().groupby("Symbol")["trade_stats"].max()
# de_lete = li_st[li_st >= 25].index.values
# 这个删除的原因？小天鹅A，外运发展等，似乎是证交所显示的停牌股票，待确认
# la_st = la_st[~la_st["Symbol"].isin(["SZ300104", "SZ300216", "SZ300028", "SZ300362"])]


la_st["tv_rank"] = la_st["daily_tv"].rank(ascending=False)
la_st["amt_rank"] = la_st["daily_amt"].rank(ascending=False)
# 2. 按照日均成交金额筛选
# data1 = all_data[all_data["Symbol"].isin(la_st["Symbol"])]
# data1 = data1[data1["ListDays"] >= 4]
# l2 = data1.groupby("Symbol")["amt"].mean().reset_index()
# l2 = l2.rename(columns={'amt': 'daily_amt'})
# data1 = data1.groupby("Symbol").last().reset_index()
# data1 = pd.merge(data1, l2, left_on="Symbol", right_on="Symbol")
# data1["amt_rank"] = data1["daily_amt"].rank(ascending=False)
# la_st = pd.merge(la_st, data1.loc[:, ["Symbol", "amt_rank"]],
#                  left_on="Symbol", right_on="Symbol", how="left")
hs = pd.concat([la_st[(la_st["HS300"] == 1) & (la_st["amt_rank"] <= len(la_st)*0.6)], la_st[(np.isnan(la_st["HS300"]))
                                            & (la_st["amt_rank"] <= len(la_st)*0.5)]])


# 3. 按照日均总市值筛选
hs["tv_rank"] = hs["daily_tv"].rank(ascending=False)
hs = hs.sort_values(by=['tv_rank'])
n_i = hs[(np.isnan(hs["HS300"])) & (hs["tv_rank"] <= 240)]
print(n_i["Symbol"].values)
print(n_i[n_i["净利润"] < 0]["Symbol"].values)
n_i = n_i[~n_i["Symbol"].isin(n_i[n_i["净利润"] < 0]["Symbol"].values)]
len1 = len(n_i)
o_i = hs[(hs["HS300"] == 1) & (hs["tv_rank"] <= 360)]
len2 = len(o_i)
if len1 <= 30:
    hs1 = n_i.iloc[:len1, :]
    if len2 >= 300 - len1:
        hs2 = o_i.iloc[:(300 - len1), :]
    elif (len2 >= 270) & (len2 < 300 - len1):
        print("Attention here!!!!!")
        hs1 = hs[(np.isnan(hs["HS300"])) & (hs["净利润"] > 0)].iloc[:(300 - len2), :]
        hs2 = o_i
    else:
        print("Attention here!!!!!")
        hs1 = hs[np.isnan(hs["HS300"])].iloc[:30, :]
        hs2 = hs[hs["HS300"] == 1].iloc[:270, :]
else:
    hs1 = n_i.iloc[:30, :]
    hs2 = hs[hs["HS300"] == 1].iloc[:270, :]


list2 = pd.concat([hs1, hs2])["Symbol"]
assert(len(list2) == 300)
dff1 = hs1.loc[:, ["Symbol", "Name", "daily_tv", "daily_amt"]]
dff1.to_csv("E:\\预测沪深调入.csv", encoding = "GBK")
dff2 = la_st1[la_st1["Symbol"].isin(list(set(old_list) - set(list2)))].loc[:,
       ["Symbol", "Name", "daily_tv", "daily_amt"]]
dff2 = dff2.sort_values(by=["daily_tv"], ascending=False)
dff2.to_csv("E:\\预测沪深调出.csv", encoding = "GBK")
li_st = list(set(hs2["Symbol"]) | set(la_st[la_st["tv_rank"] <= 300]["Symbol"]) | set(hs1["Symbol"]))
data2 = la_st.drop(la_st[la_st["Symbol"].isin(li_st)].index, axis=0)

dff2 = list(set(old_list) - set(list2))

print("沪深300预测新入股票数 %d" % len(set(dff1["Symbol"])))
print("沪深300实际新入股票数 %d" % len(set(set(new_list) - set(old_list))))
print("沪深300新入预测与实际相同股票数 %d" % len(set(set(new_list) - set(old_list)) & set(dff1["Symbol"])))
print("沪深300预测旧出股票数 %d" % len(dff2))
print("沪深300实际旧出股票数 %d" % len(set(set(old_list) - set(new_list))))
print("沪深300旧出预测与实际相同股票数 %d" % len(set(set(old_list) - set(new_list)) & set(dff2)))
print("沪深300预测与实际相同股票数 %d" % len(set(new_list) & set(list2)))

# 第二部分：筛选中证500样本指数成分股
data2["tv_rank"] = data2["daily_tv"].rank(ascending=False)
data2["amt_rank"] = data2["daily_amt"].rank(ascending=False)
# 1. 按照日均成交金额筛选
data2 = pd.concat([data2[(data2["CSI500"] == 1) & (data2["amt_rank"] < len(data2)*0.9)],
                   data2[(np.isnan(data2["CSI500"])) & (data2["amt_rank"] < len(data2)*0.8)]])
# 2. 按照日均总市值筛选
data2["tv_rank"] = data2["daily_tv"].rank(ascending=False)
data2 = data2.sort_values(by=['tv_rank'])
n_i = data2[(np.isnan(data2["CSI500"])) & (data2["tv_rank"] <= 400)]
print(n_i["Symbol"].values)
print(n_i[n_i["净利润"] < 0]["Symbol"].values)
n_i = n_i[~n_i["Symbol"].isin(n_i[n_i["净利润"] < 0]["Symbol"].values)]
o_i = data2[(data2["CSI500"] == 1) & (data2["tv_rank"] <= 600)]
len3 = len(n_i)
len4 = len(o_i)

if len3 <= 50:
    db1 = n_i.iloc[:len3, :]
    if len4 >= 500 - len3:
        db2 = o_i.iloc[:(500 - len3), :]
    elif (len4 >= 450) & (len4 < 500 - len3):
        print("Attention here!!!!!")
        db1 = data2[data2["CSI500"] != 1].iloc[:(500 - len4), :]
        db2 = o_i
    else:
        print("Attention here!!!!!")
        db1 = data2[data2["CSI500"] != 1].iloc[:50, :]
        db2 = data2[data2["CSI500"] == 1].iloc[:450, :]
else:
    db1 = n_i.iloc[:50, :]
    db2 = data2[data2["CSI500"] == 1].iloc[:450, :]

print(db1[db1["净利润"] < 0]["Symbol"].values)

new_list = pd.concat([db1, db2])["Symbol"]
assert(len(new_list) == 500)
dff1 = db1.loc[:, ["Symbol", "Name", "daily_tv", "daily_amt"]]
dff1 = dff1.sort_values(by=["daily_tv"], ascending=False)
dff1.to_csv("E:\\预测中证调入.csv", encoding = "GBK")
dff2 = la_st1[la_st1["Symbol"].isin(list(set(total_list) - set(new_list)))].loc[:,
       ["Symbol", "Name", "daily_tv", "daily_amt"]]
dff2 = dff2.sort_values(by=["daily_tv"], ascending=False)
dff2.to_csv("E:\\预测中证调出.csv", encoding = "GBK")

# dff2 = list(set(total_list) - set(new_list))

# print("中证500新入预测与实际相同股票数 %d" % len(set(set(total_list1) - set(total_list)) & set(dff1["Symbol"])))
# print("中证500旧出预测与实际相同股票数 %d" % len(set(set(total_list) - set(total_list1)) & set(dff2)))
# print("中证500预测与实际相同股票数 %d" % len(set(total_list1) & set(new_list)))


# 第三部分：筛选中证1000样本指数成分股
nd = la_st1[(la_st1["ListDays"] > 60) &
             (la_st1["Name"].str.contains("ST") == False) & (la_st1["Symbol"].str[:5] != "SH688") &
             (la_st1["Name"].str.contains("退") == False)]
nd = nd[~nd["Symbol"].isin(["SZ300104", "SZ300216"])]

nd["tv_rank"] = nd["daily_tv"].rank(ascending=False)
nd["amt_rank"] = nd["daily_amt"].rank(ascending=False)
li_st = list(set(list2) | set(nd[nd["tv_rank"] <= 300]["Symbol"]) | set(new_list))
data3_1 = nd[~nd["Symbol"].isin(li_st)]

data3_1["tv_rank"] = data3_1["daily_tv"].rank(ascending=False)
data3_1["amt_rank"] = data3_1["daily_amt"].rank(ascending=False)
# 1. 按照日均成交金额筛选
data3 = pd.concat([data3_1[(data3_1["CSI1000"] == 1) & (data3_1["amt_rank"] < len(data3_1)*0.9)],
                   data3_1[(np.isnan(data3_1["CSI1000"])) & (data3_1["amt_rank"] < len(data3_1)*0.8)]])
# 2. 按照日均总市值筛选
data3["tv_rank"] = data3["daily_tv"].rank(ascending=False)
data3 = data3.sort_values(by=['tv_rank'])
n_i = data3[(np.isnan(data3["CSI1000"])) & (data3["tv_rank"] <= 800)]
print(n_i["Symbol"].values)
print(n_i[n_i["净利润"] < 0]["Symbol"].values)
n_i = n_i[~n_i["Symbol"].isin(n_i[n_i["净利润"] < 0]["Symbol"].values)]
o_i = data3[(data3["CSI1000"] == 1) & (data3["tv_rank"] <= 1200)]
len5 = len(n_i)
len6 = len(o_i)

if len5 <= 100:
    db1 = n_i.iloc[:len5, :]
    if len6 >= 1000 - len5:
        db2 = o_i.iloc[:(1000 - len5), :]
    elif (len6 >= 900) & (len6 < 1000 - len5):
        print("Attention here!!!!!")
        db1 = data3[data3["CSI1000"] != 1].iloc[:(1000 - len6), :]
        db2 = o_i
    else:
        print("Attention here!!!!!")
        db1 = data3[data3["CSI1000"] != 1].iloc[:100, :]
        db2 = data3[data3["CSI1000"] == 1].iloc[:900, :]
else:
    db1 = n_i.iloc[:100, :]
    db2 = data3[data3["CSI1000"] == 1].iloc[:900, :]

print(db1[db1["净利润"] < 0]["Symbol"].values)

new_list1 = pd.concat([db1, db2])["Symbol"]
dff1 = db1.loc[:, ["Symbol", "Name", "daily_tv", "daily_amt"]]
dff1 = dff1.sort_values(by=["daily_tv"], ascending=False)
dff1.to_csv("E:\\预测中证1000调入.csv", encoding = "GBK")
dff2 = la_st1[la_st1["Symbol"].isin(list(set(ol) - set(new_list1)))].loc[:,
       ["Symbol", "Name", "daily_tv", "daily_amt"]]
dff2 = dff2.sort_values(by=["daily_tv"], ascending=False)
dff2.to_csv("E:\\预测中证1000调出.csv", encoding = "GBK")

dff2 = list(set(ol) - set(new_list1))

# ni_list = list(set(nl) - set(ol))
# oo_list = list(set(ol) - set(nl))
# if "SZ001914" in ni_list:
#     ni_list.remove("SZ001914")
#     oo_list.remove("SZ000043")

print("中证1000预测新入股票数 %d" % len(set(dff1["Symbol"])))
# print("中证1000实际新入股票数 %d" % len(ni_list))
# print("中证1000新入预测与实际相同股票数 %d" % len(set(ni_list) & set(dff1["Symbol"])))
print("中证1000预测旧出股票数 %d" % len(set(dff2)))
# print("中证1000实际旧出股票数 %d" % len(oo_list))      
# print("中证1000旧出预测与实际相同股票数 %d" % len(set(oo_list) & set(dff2)))
# print("中证1000预测与实际相同股票数 %d" % len(set(nl) & set(new_list1)))

# # 第四部分：筛选中证全指调出
# sto_ck = pd.read_csv('F:\\all_stock_202004_1.csv', encoding = "GBK").iloc[:,1:]
# sto_ck["returns"] = sto_ck.groupby("Symbol")['close'].apply(lambda x: x/x.shift(1)-1)
# sto_ck["Date"] = sto_ck["Date"].apply(lambda x: str(x))
# sto_ck["Date"] = pd.to_datetime(sto_ck["Date"])
# st_list = sto_ck[(sto_ck["Name"].str.contains("ST") != False)  |
#              (sto_ck["Name"].str.contains("退") != False)]["Symbol"].unique()
# out_bottom = list(set(ol1) & set(st_list))
# out_top = list((set(ol1) & set(new_list1)) | (set(ol1) & set(new_list)) | (set(ol1) & set(list2)))
# o_ut = set(out_bottom) | set(out_top)
# # oo_list1 = list(set(ol1) - set(nl1))

# dff2 = la_st1[la_st1["Symbol"].isin(o_ut)].loc[:,
#        ["Symbol", "Name", "daily_tv", "daily_amt"]]
# dff2 = dff2.sort_values(by=["daily_tv"], ascending=False)
# dff2.to_csv("E:\\预测中证全指调出.csv", encoding = "GBK")
# print("中证全指预测旧出股票数 %d" % len(o_ut))
# # print("中证全指实际旧出股票数 %d" % len(oo_list1))      
# # print("中证全指旧出预测与实际相同股票数 %d" % len(set(oo_list1) & set(o_ut)))

In [41]:
sto_ck[(sto_ck["Date"] == '2020-04-30') & (sto_ck["Symbol"].isin(st_list)) & (sto_ck["Name"].str.contains("ST") == False) &
             (sto_ck["Name"].str.contains("退") == False)]

,Date,Symbol,Name,ListDays,Totalshares,Marketshares,TotalValue,MarketValue,close,open,ret,volume,amt,returns
37262,2020-04-30,SH600186,莲花健康,5005,1.379924e+09,1.379924e+09,3.932783e+05,3.932783e+05,2.85,2.87,0.352113,10324862,2.935720e+07,0.003521
65594,2020-04-30,SH600321,正源股份,4427,1.510550e+09,1.510550e+09,3.202366e+05,3.202366e+05,2.12,2.05,1.923077,13091788,2.771274e+07,0.019231
80933,2020-04-30,SH600396,金山股份,4471,1.472707e+09,1.472707e+09,3.711221e+05,3.711221e+05,2.52,2.51,0.800000,30365186,7.683451e+07,0.008000
103059,2020-04-30,SH600526,菲达环保,4241,5.474047e+08,5.474047e+08,2.835556e+05,2.835556e+05,5.18,5.09,2.777778,6553142,3.370066e+07,0.027778
108899,2020-04-30,SH600556,天下秀,2884,1.680420e+09,3.487537e+08,2.802941e+06,5.817211e+05,16.68,16.45,0.542495,24379204,4.083094e+08,0.005425
122053,2020-04-30,SH600615,丰华股份,6299,1.880205e+08,1.868100e+08,1.489122e+05,1.479535e+05,7.92,7.86,0.891720,1581001,1.252817e+07,0.008917
147365,2020-04-30,SH600732,爱旭股份,5099,1.829888e+09,4.463831e+08,1.414504e+06,3.450541e+05,7.73,7.40,3.480589,25643508,1.971193e+08,0.034806
230194,2020-04-30,SH601798,蓝科高新,2085,3.545282e+08,3.545282e+08,2.123624e+05,2.123624e+05,5.99,6.00,0.841751,1849300,1.105048e+07,0.008418
336173,2020-04-30,SH603810,丰山集团,390,8.300500e+07,3.951181e+07,2.862842e+05,1.362762e+05,34.49,33.70,0.759568,2922202,9.922849e+07,0.007596
381564,2020-04-30,SZ000048,京基智农,5935,4.024917e+08,4.024287e+08,1.006632e+06,1.006474e+06,25.01,25.58,-6.434718,7143871,1.793877e+08,-0.064347


In [34]:
sto_ck[(sto_ck["Symbol"] == 'SZ002194') & (sto_ck["Name"] == "*ST凡谷")]

,Date,Symbol,Name,ListDays,Totalshares,Marketshares,TotalValue,MarketValue,close,open,ret,volume,amt
530403,20190506,SZ002194,*ST凡谷,2716,564669722.0,411336539.0,1.018664e+06,742051.116356,18.04,18.04,-5.002633,308600,5.567144e+06
530404,20190507,SZ002194,*ST凡谷,2717,564669722.0,411336539.0,9.678439e+05,705030.827846,17.14,17.14,-4.988914,14089670,2.430333e+08
530405,20190508,SZ002194,*ST凡谷,2718,564669722.0,411336539.0,9.192823e+05,669655.885492,16.28,16.28,-5.017503,16871347,2.750124e+08
530406,20190509,SZ002194,*ST凡谷,2719,564669722.0,411336539.0,8.735441e+05,636337.625833,15.47,15.55,-4.975430,4675000,7.266088e+07
530407,20190510,SZ002194,*ST凡谷,2720,564669722.0,411336539.0,9.170236e+05,668010.539336,16.24,15.01,4.977376,32726299,5.088861e+08
530408,20190513,SZ002194,*ST凡谷,2721,564669722.0,411336539.0,8.763674e+05,638394.308528,15.52,15.70,-4.433498,16519920,2.608636e+08
530409,20190514,SZ002194,*ST凡谷,2722,564669722.0,411336539.0,8.554746e+05,623174.856585,15.15,15.01,-2.384021,14938247,2.274678e+08
530410,20190515,SZ002194,*ST凡谷,2723,564669722.0,411336539.0,8.549100e+05,622763.520046,15.14,15.40,-0.066007,13122642,2.003067e+08
530411,20190516,SZ002194,*ST凡谷,2724,564669722.0,411336539.0,8.119951e+05,591501.943082,14.38,15.14,-5.019815,17920550,2.590175e+08
530412,20190517,SZ002194,*ST凡谷,2725,564669722.0,411336539.0,8.323232e+05,606310.058486,14.74,14.28,2.503477,19878464,2.924214e+08


In [69]:
len(data3_1)*0.9

2405.7000000000003

In [106]:
data3[(data3["CSI1000"] == 1) & (data3["净利润"] < 0)].sort_values(by="净利润")

,Symbol,Date,Name,TotalValue,MarketValue,Totalshares,Marketshares,close,amt,ListDays,净利润,CSI500,HS300,CSI1000,daily_tv,daily_amt,TradeDays,tv_rank,amt_rank
2900,SZ300064,2020-04-30,豫金刚石,2.459172e+09,1.235536e+05,1.205477e+09,6.056551e+08,2.04,3.840771e+07,2350,-5.199212e+09,NaN,NaN,1.0,4.317681e+09,3.603690e+07,244,1124.0,1987.0
1727,SZ000662,2020-04-30,天夏智慧,3.279293e+09,3.278942e+05,1.093098e+09,1.092981e+09,3.00,3.030398e+08,5230,-4.008154e+09,NaN,NaN,1.0,5.200368e+09,1.844432e+08,244,864.0,338.0
183,SH600239,2020-04-30,云南城投,4.592265e+09,4.592265e+05,1.605687e+09,1.605687e+09,2.86,1.306745e+08,4397,-3.482046e+09,NaN,NaN,1.0,4.772641e+09,7.026913e+07,244,981.0,1244.0
3199,SZ300367,2020-04-30,东方网力,3.587318e+09,2.505182e+05,1.195773e+09,8.350606e+08,3.00,7.070235e+07,1338,-3.174361e+09,NaN,NaN,1.0,6.142956e+09,1.474676e+08,244,642.0,496.0
547,SH600734,2020-04-30,实达集团,1.935578e+09,1.614689e+05,6.223723e+08,5.191926e+08,3.11,6.411107e+07,4743,-3.051483e+09,NaN,NaN,1.0,4.576998e+09,1.939664e+08,244,1043.0,303.0
3055,SZ300222,2020-04-30,科大智能,5.725530e+09,3.679807e+05,7.247506e+08,4.657984e+08,7.90,5.131311e+07,1854,-2.631885e+09,NaN,NaN,1.0,7.670010e+09,1.120926e+08,244,379.0,721.0
2506,SZ002619,2020-04-30,艾格拉斯,4.187914e+09,3.225244e+05,1.844896e+09,1.420812e+09,2.27,4.662673e+07,1669,-2.555990e+09,NaN,NaN,1.0,5.619751e+09,8.254915e+07,244,760.0,1047.0
721,SH600986,2020-04-30,科达股份,5.168250e+09,5.114629e+05,1.325192e+09,1.311443e+09,3.90,1.187808e+08,3651,-2.503359e+09,NaN,NaN,1.0,5.764429e+09,2.465483e+08,244,725.0,197.0
2139,SZ002219,2020-04-29,恒康医疗,3.320121e+09,3.282656e+05,1.865236e+09,1.844189e+09,1.78,4.509333e+07,2474,-2.497510e+09,NaN,NaN,1.0,4.965597e+09,5.363490e+07,243,923.0,1554.0
2428,SZ002537,2020-04-30,海联金汇,7.649822e+09,7.649822e+05,1.237835e+09,1.237835e+09,6.18,2.770414e+08,2109,-2.456925e+09,NaN,NaN,1.0,9.343548e+09,2.983933e+08,244,236.0,128.0


In [102]:
data3[(data3["CSI1000"] == 1)].tail(50)

,Symbol,Date,Name,TotalValue,MarketValue,Totalshares,Marketshares,close,amt,ListDays,净利润,CSI500,HS300,CSI1000,daily_tv,daily_amt,TradeDays,tv_rank,amt_rank
1219,SH603612,2020-04-30,索通发展,3.592280e+09,190667.030580,3.369869e+08,1.788621e+08,10.66,4.499297e+07,680,8.883720e+07,NaN,NaN,1.0,3.767207e+09,5.279696e+07,244,1340.0,1576.0
3192,SZ300359,2020-04-30,全通教育,3.424158e+09,288114.867540,6.341034e+08,5.335461e+08,5.40,4.664312e+07,1360,-7.165140e+08,NaN,NaN,1.0,3.753554e+09,7.415658e+07,244,1345.0,1171.0
2625,SZ002751,2020-04-30,易尚展示,3.137377e+09,238246.293180,1.545506e+08,1.173627e+08,20.30,6.551771e+07,1182,7.628193e+07,NaN,NaN,1.0,3.742493e+09,1.699718e+08,244,1350.0,390.0
2362,SZ002464,2020-04-30,众应互联,3.402099e+09,340180.976528,5.217944e+08,5.217500e+08,6.52,4.953936e+08,1768,-1.353180e+09,NaN,NaN,1.0,3.734840e+09,2.064369e+08,244,1352.0,271.0
2938,SZ300102,2020-04-30,乾照光电,3.226272e+09,322620.369816,7.075158e+08,7.075008e+08,4.56,1.076585e+08,2269,-2.800072e+08,NaN,NaN,1.0,3.730649e+09,1.068431e+08,244,1354.0,768.0
1698,SZ000615,2020-04-30,京汉股份,3.152700e+09,302045.021304,7.823077e+08,7.494914e+08,4.03,4.200110e+07,5399,1.239201e+08,NaN,NaN,1.0,3.729573e+09,4.778460e+07,243,1355.0,1700.0
2151,SZ002232,2020-04-30,启明信息,3.774988e+09,373030.274232,4.085485e+08,4.037124e+08,9.24,6.951506e+07,2891,9.471798e+07,NaN,NaN,1.0,3.729562e+09,8.814123e+07,244,1356.0,978.0
1639,SZ000532,2020-04-30,华金资本,3.698720e+09,368557.737558,3.447083e+08,3.434834e+08,10.73,1.690166e+08,6226,7.924529e+07,NaN,NaN,1.0,3.724235e+09,1.140999e+08,244,1358.0,707.0
1916,SZ000965,2020-04-30,天保基建,3.618049e+09,361804.871770,1.109831e+09,1.109831e+09,3.26,4.714974e+07,4672,1.898612e+08,NaN,NaN,1.0,3.720390e+09,2.918799e+07,244,1361.0,2167.0
3165,SZ300332,2020-04-30,天壕环境,4.189756e+09,397648.513136,8.802009e+08,8.353960e+08,4.76,1.421084e+08,1683,1.061367e+07,NaN,NaN,1.0,3.717189e+09,4.999804e+07,244,1364.0,1642.0


In [112]:
data3[data3["Symbol"].isin(['SZ000552', 'SZ000687', 'SZ000828', 'SZ000892', 'SZ002293', 'SZ002534',
             'SZ002650', 'SZ002721', 'SZ300064', 'SZ300128', 'SZ300317', 'SZ300343', 'SZ300367', 'SZ300467',
             'SH600146', 'SH600172', 'SH600180', 'SH600227', 'SH600382', 'SH600734', 'SH600790',
             'SH601566', 'SH603096'])].sort_values(by=["净利润", "amt_rank"]).to_csv(r"D:\work\project 3 event study\CSI1000_outToBottom_diff.csv", encoding="GBK", index=False)



In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
from datetime import timedelta
import calendar
import seaborn as sns
import datetime

csi_index = pd.read_csv('D:\\work\\project 3 event study\\index_comp_SH000905.csv', encoding="utf-8")
csi_index_300 = pd.read_csv('D:\\work\\project 3 event study\\index_comp_SH000300.csv', encoding="utf-8")
csi_index_1000 = pd.read_csv('D:\\work\\project 3 event study\\index_comp_SH000852.csv', encoding="utf-8")
csi_rest = pd.read_csv('D:\\work\\project 3 event study\\index_comp_SH000985.csv', encoding="utf-8")
hs300_list = csi_index_300.columns
zz500_list = csi_index.columns
zz1000_list = csi_index_1000.columns
zzall_list = csi_rest.columns
real_date = csi_rest["Date"].max()
rest1 = zzall_list[(csi_rest[csi_rest["Date"] == real_date] != 0).values[0]][1:]
zz10001 = zz1000_list[(csi_index_1000[csi_index_1000["Date"] == real_date] != 0).values[0]][1:]
zz5001 = zz500_list[(csi_index[csi_index["Date"] == real_date] != 0).values[0]][1:]
hs3001 = hs300_list[(csi_index_300[csi_index_300["Date"] == real_date] != 0).values[0]][1:]
display(len(hs3001))
display(len(zz5001))
display(len(zz10001))
display(len(rest1))
IF_in = pd.read_csv('D:\\work\\project 4 event study - minute\\IF_in_202006.csv', encoding="utf-8")
IF_out = pd.read_csv('D:\\work\\project 4 event study - minute\\IF_out_202006.csv', encoding="utf-8")
hs300 = list((set(hs3001) | set(IF_in["StockID"])) - set(IF_out["StockID"]))
IC_in = pd.read_csv('D:\\work\\project 4 event study - minute\\IC_in_202006.csv', encoding="utf-8")
IC_out = pd.read_csv('D:\\work\\project 4 event study - minute\\IC_out_202006.csv', encoding="utf-8")
zz500 = list((set(zz5001) | set(IC_in["StockID"])) - set(IC_out["StockID"]))
CSI1000_in = pd.read_csv('D:\\work\\project 4 event study - minute\\CSI1000_in_202006.csv', encoding="utf-8")
CSI1000_out = pd.read_csv('D:\\work\\project 4 event study - minute\\CSI1000_out_202006.csv', encoding="utf-8")
zz1000 = list((set(zz10001) | set(CSI1000_in["StockID"])) - set(CSI1000_out["StockID"]))
CSIall_in = pd.read_csv('D:\\work\\project 4 event study - minute\\CSIRest_in_202006.csv', encoding="utf-8")
CSIall_out = pd.read_csv('D:\\work\\project 4 event study - minute\\CSIRest_out_202006.csv', encoding="utf-8")
zzall = list(((set(rest1) | set(zz10001) | set(zz5001) | set(hs3001) )| set(CSIall_in["StockID"])) - set(CSIall_out["StockID"]))
rest1 = list(((set(rest1) - set(zz10001)) - set(zz5001)) - set(hs3001))
rest2 = list(((set(zzall) - set(zz1000)) - set(zz500)) - set(hs300))
df = pd.DataFrame()
df["StockID"]  = rest2
df.to_csv('D:\\work\\project 4 event study - minute\\CSIRest.csv')
CSIrest_out = list(set(rest1) - set(rest2))
df = pd.DataFrame()
df["StockID"]  = CSIrest_out
df.to_csv('D:\\work\\project 4 event study - minute\\new\\CSIRest_out_202006.csv')
CSIrest_in = list(set(rest2) - set(rest1))
df = pd.DataFrame()
df["StockID"]  = CSIrest_in
df.to_csv('D:\\work\\project 4 event study - minute\\new\\CSIRest_in_202006.csv')

300

500

1000

1666

In [42]:
'SZ002600' in CSIrest_in

False

In [26]:
len(CSIall_out)

113

In [13]:
rest2

['SZ300340',
 'SZ002272',
 'SZ002132',
 'SH600463',
 'SZ300693',
 'SZ300164',
 'SZ300196',
 'SZ300561',
 'SH603232',
 'SZ300112',
 'SZ300711',
 'SZ300390',
 'SZ300432',
 'SZ000949',
 'SZ300331',
 'SH600218',
 'SZ000833',
 'SH600814',
 'SH600713',
 'SZ002088',
 'SH603729',
 'SZ300807',
 'SH603416',
 'SH603906',
 'SZ002213',
 'SH603689',
 'SZ002932',
 'SH603133',
 'SZ300479',
 'SZ300414',
 'SZ300729',
 'SZ300480',
 'SZ002790',
 'SZ002403',
 'SH600235',
 'SZ300417',
 'SH600620',
 'SZ002742',
 'SZ000715',
 'SH600735',
 'SZ300683',
 'SZ000839',
 'SZ002606',
 'SZ002119',
 'SH600469',
 'SZ000524',
 'SZ002671',
 'SZ002350',
 'SZ300135',
 'SZ300710',
 'SH600802',
 'SH600600',
 'SZ002861',
 'SZ300500',
 'SH603813',
 'SZ000554',
 'SZ300519',
 'SZ002211',
 'SZ002441',
 'SH603505',
 'SZ300668',
 'SH603186',
 'SH603177',
 'SZ002347',
 'SZ300545',
 'SH600861',
 'SZ300655',
 'SH603181',
 'SH603757',
 'SH600586',
 'SH603336',
 'SZ002330',
 'SZ002362',
 'SZ300617',
 'SZ002816',
 'SZ000404',
 'SH601007',

In [3]:
pd.set_option("max_rows", 100)
data2[data2["CSI500"] == 1].iloc[449:]

,Symbol,Date,Name,TotalValue,MarketValue,Totalshares,Marketshares,close,amt,ListDays,净利润,CSI500,HS300,CSI1000,daily_tv,daily_amt,TradeDays,tv_rank,amt_rank
568,SH600757,2020-04-30,长江传媒,6.298845e+09,629804.996457,1.213650e+09,1.213497e+09,5.19,3.826878e+07,5290,8.034538e+08,1.0,NaN,NaN,7.520105e+09,4.571373e+07,244,847.0,2052.0
3031,SZ300197,2020-04-30,铁汉生态,6.335091e+09,478778.332140,2.346330e+09,1.773253e+09,2.70,6.357988e+07,1901,-9.222546e+08,1.0,NaN,NaN,7.484276e+09,7.585579e+07,239,853.0,1419.0
2840,SZ300002,2020-04-30,神州泰岳,1.025651e+10,740060.804014,1.961092e+09,1.415030e+09,5.23,7.167852e+08,2434,-1.469479e+09,1.0,NaN,NaN,7.484138e+09,2.011377e+08,244,854.0,425.0
579,SH600770,2020-04-30,综艺股份,6.396000e+09,639600.000000,1.300000e+09,1.300000e+09,4.92,1.021028e+08,5541,1.465050e+08,1.0,NaN,NaN,7.309570e+09,1.013153e+08,244,872.0,1068.0
2337,SZ002437,2020-04-30,誉衡药业,6.440500e+09,637212.203947,2.198123e+09,2.174786e+09,2.93,1.248903e+08,2127,-2.654938e+09,1.0,NaN,NaN,7.308849e+09,1.264447e+08,244,873.0,840.0
2263,SZ002358,2020-04-30,森源电气,5.150854e+09,504375.189620,9.297570e+08,9.104245e+08,5.54,3.433082e+07,2265,1.351368e+07,1.0,NaN,NaN,7.245779e+09,9.100938e+07,244,883.0,1213.0
2463,SZ002573,2020-04-30,清新环境,6.022686e+09,600738.773125,1.081272e+09,1.078526e+09,5.57,2.039005e+07,1949,3.773525e+08,1.0,NaN,NaN,7.221258e+09,4.529728e+07,244,888.0,2057.0
3130,SZ300297,2020-04-30,蓝盾股份,4.861746e+09,387943.792257,1.249806e+09,9.972848e+08,3.89,1.547465e+08,1847,-9.101757e+08,1.0,NaN,NaN,7.087570e+09,1.711049e+08,244,904.0,554.0
734,SH601001,2020-04-30,大同煤业,6.142479e+09,614247.900000,1.673700e+09,1.673700e+09,3.67,6.881540e+07,3186,1.366946e+09,1.0,NaN,NaN,7.029197e+09,3.942729e+07,244,913.0,2211.0
2379,SZ002482,2020-04-30,广田集团,5.242124e+09,521489.457236,1.537280e+09,1.529295e+09,3.41,1.506629e+07,2278,1.325997e+08,1.0,NaN,NaN,7.028518e+09,3.972222e+07,244,914.0,2200.0


In [5]:
la_st1[la_st1["Symbol"] == "SH601019"]

,Symbol,Date,Name,TotalValue,MarketValue,Totalshares,Marketshares,close,amt,ListDays,净利润,CSI500,HS300,CSI1000,daily_tv,daily_amt,TradeDays
748,SH601019,2020-04-30,山东出版,1.310573e+10,268093.2,2.086900e+09,426900000.0,6.28,18608149.0,594,1.541437e+09,1.0,NaN,NaN,1.474369e+10,2.602632e+07,244


In [35]:
dff2 = pd.read_csv(r"E:\预测中证调出.csv", encoding = "GBK")
len(set(total_list) - set(new_list) - set(dff2["Symbol"].values))

4

In [36]:
sto_ck[sto_ck["Symbol"].isin(set(total_list) - set(new_list) - set(dff2["Symbol"].values))].groupby(["Symbol"])["Name"].last()

Symbol
SZ000536    *ST华映
SZ000727    *ST东科
SZ002176    *ST江特
SZ002426    *ST胜利
Name: Name, dtype: object

In [32]:
for i in ['SZ300431', 'SZ300362', 'SZ300345', 'SZ300312', 'SZ300234', 'SZ300216', 
             'SZ300167', 'SZ300163', 'SZ300156', 'SZ300116', 'SZ300104', 'SZ300093', 
             'SZ300090', 'SZ300089', 'SZ300080', 'SZ300077', 'SZ300071', 'SZ300063', 
             'SZ300032', 'SZ300008', 'SZ300005']:
    if i in ol1:
        print(i)

SZ300431
SZ300008


In [59]:
la_st1[la_st1["Symbol"].isin(nl[nl.str[:3] == "SZ3"].values)]["ListDays"].min()

73

In [48]:
ts[ts["证券代码"] == "SH600682"]

,证券代码,证券简称,A股合计,B股合计,H股合计
2784,SH600682,南京新百,1.346132e+09,0.0,0.0


In [47]:
sto_ck[sto_ck["Symbol"] == "SH600682"]

,Date,Symbol,Name,ListDays,Totalshares,Marketshares,TotalValue,MarketValue,close,open,...,amt,returns,证券代码_x,证券简称,A股合计,B股合计,H股合计,证券代码_y,证券名称,净利润
124548,2018-11-01,SH600682,南京新百,5592,1.292713e+09,9.730801e+08,1.234541e+10,9.292915e+05,9.55,9.44,...,155201790.0,NaN,SH600682,南京新百,1.346132e+09,0.0,0.0,SH600682,南京新百,1.075475e+09
124549,2018-11-02,SH600682,南京新百,5593,1.292713e+09,9.730801e+08,1.273323e+10,9.584839e+05,9.85,9.65,...,160794122.0,0.031414,SH600682,南京新百,1.346132e+09,0.0,0.0,SH600682,南京新百,1.075475e+09
124550,2018-11-05,SH600682,南京新百,5594,1.292713e+09,9.730801e+08,1.312104e+10,9.876763e+05,10.15,9.91,...,194364939.0,0.030457,SH600682,南京新百,1.346132e+09,0.0,0.0,SH600682,南京新百,1.075475e+09
124551,2018-11-06,SH600682,南京新百,5595,1.292713e+09,9.730801e+08,1.317275e+10,9.915686e+05,10.19,10.04,...,118188092.0,0.003941,SH600682,南京新百,1.346132e+09,0.0,0.0,SH600682,南京新百,1.075475e+09
124552,2018-11-07,SH600682,南京新百,5596,1.292713e+09,9.730801e+08,1.331495e+10,1.002272e+06,10.30,10.25,...,229577588.0,0.010795,SH600682,南京新百,1.346132e+09,0.0,0.0,SH600682,南京新百,1.075475e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124783,2019-10-25,SH600682,南京新百,5827,1.346132e+09,1.002157e+09,1.390555e+10,1.035228e+06,10.33,10.26,...,70295141.0,-0.003857,SH600682,南京新百,1.346132e+09,0.0,0.0,SH600682,南京新百,1.075475e+09
124784,2019-10-28,SH600682,南京新百,5828,1.346132e+09,1.002157e+09,1.426900e+10,1.062286e+06,10.60,10.26,...,135569567.0,0.026137,SH600682,南京新百,1.346132e+09,0.0,0.0,SH600682,南京新百,1.075475e+09
124785,2019-10-29,SH600682,南京新百,5829,1.346132e+09,1.002157e+09,1.436323e+10,1.069301e+06,10.67,10.57,...,126092747.0,0.006604,SH600682,南京新百,1.346132e+09,0.0,0.0,SH600682,南京新百,1.075475e+09
124786,2019-10-30,SH600682,南京新百,5830,1.346132e+09,1.002157e+09,1.472669e+10,1.096359e+06,10.94,10.68,...,190042157.0,0.025305,SH600682,南京新百,1.346132e+09,0.0,0.0,SH600682,南京新百,1.075475e+09


In [46]:
"SH600682" in csi_rest.columns[(csi_rest[csi_rest["Date"] == 20200102]!=0).values[0]][1:]

True

In [3]:
stock1 = pd.read_csv('F:\\all_stock_201910.csv', encoding = "GBK").iloc[:,1:]
stock2 = pd.read_csv('F:\\all_stock_201910_1.csv', encoding = "GBK").iloc[:,1:]
sl1 = stock1[(stock1["Name"].str.contains("ST") == True)  |
             (stock1["Name"].str.contains("退") == True)]["Symbol"].unique()
sl2 = stock2[(stock2["Name"].str.contains("ST") == True)  |
             (stock2["Name"].str.contains("退") == True)]["Symbol"].unique()
print(set(sl2) - set(sl1))
display(list(set(sl2) - set(sl1))[0] in list2.values)
display(list(set(sl2) - set(sl1))[0] in new_list.values)
display(list(set(sl2) - set(sl1))[0] in new_list1.values)

{'SH600319'}


False

False

False

In [137]:
hs[hs["Symbol"].isin(set(set(new_list) - set(old_list)) - set(hs1["Symbol"]))]

,Symbol,Date,Name,TotalValue,MarketValue,Totalshares,Marketshares,close,amt,ListDays,净利润,CSI500,HS300,daily_tv_x,daily_amt_x,TradeDays,tv_rank,amt_rank_x,daily_amt_y,amt_rank_y,daily_tv_y
1823,SZ002001,2018-10-31,新 和 成,3.029614e+10,2.572004e+06,2.148662e+09,1.824116e+09,14.10,1.695793e+08,3433,2.067594e+09,NaN,NaN,2.953411e+10,6.328789e+08,244,258.0,80.0,6.328789e+08,80.0,2.953411e+10
1369,SH603986,2018-10-24,兆易创新,1.962339e+10,1.429796e+06,2.846445e+08,2.073972e+08,68.94,4.220317e+08,330,2.348295e+08,NaN,NaN,2.828901e+10,7.687635e+08,158,277.0,59.0,7.687635e+08,59.0,2.828901e+10
3003,SZ300296,2018-10-31,利亚德,1.937672e+10,1.252883e+06,2.542877e+09,1.644204e+09,7.62,3.646902e+08,1435,5.864266e+08,NaN,NaN,2.661817e+10,2.481896e+08,244,301.0,338.0,2.481896e+08,340.0,2.661817e+10
2080,SZ002271,2018-10-31,东方雨虹,2.022370e+10,1.311244e+06,1.499162e+09,9.720116e+08,13.49,2.806102e+08,2437,6.172604e+08,NaN,NaN,2.112928e+10,2.430878e+08,244,393.0,353.0,2.430878e+08,355.0,2.112928e+10


In [162]:
sto_ck = pd.read_csv('F:\\all_stock_201810.csv', encoding = "GBK").iloc[:,1:]
test = sto_ck[sto_ck["Symbol"] == "SZ002001"]
close = pd.read_csv("F:\\kkk.csv", encoding="GBK").iloc[:, :4]
close["日期"] = close["日期"].apply(lambda x: int(x.split('-')[0] + x.split('-')[1] + x.split('-')[2]))
test = pd.merge(test, close, left_on="Date", right_on="日期")
test["TotalValue"] = test["Totalshares"] * test["收盘价(元)"]
test["TotalValue"].mean()

28528569602.939648

In [120]:
set(set(total_list1) - set(total_list)) - set(dff1["Symbol"])

{'SH600507',
 'SH600699',
 'SH601100',
 'SZ000060',
 'SZ002470',
 'SZ300316',
 'SZ300413',
 'SZ300450',
 'SZ300459'}

In [121]:
sto_ck[sto_ck["Symbol"] == "SH600507"]

,Date,Symbol,Name,ListDays,Totalshares,Marketshares,TotalValue,MarketValue,close,open,ret,volume,amt,returns,证券代码_x,证券简称,A股合计,B股合计,H股合计,证券代码_y,证券名称,净利润
89446,2017-11-01,SH600507,方大特钢,3313,1.326093e+09,1.326093e+09,1.522326e+10,1.686790e+06,11.479781,11.624180,-0.625000,2.059424e+07,238461902.0,NaN,SH600507,方大特钢,1.449871e+09,0.0,0.0,SH600507,方大特钢,1.312697e+09
89447,2017-11-02,SH600507,方大特钢,3314,1.326093e+09,1.326093e+09,1.513948e+10,1.677508e+06,11.416606,11.443681,-0.550314,1.419502e+07,162736701.0,-0.005503,SH600507,方大特钢,1.449871e+09,0.0,0.0,SH600507,方大特钢,1.312697e+09
89448,2017-11-03,SH600507,方大特钢,3315,1.326093e+09,1.326093e+09,1.479241e+10,1.639051e+06,11.154881,11.371481,-2.292490,2.315736e+07,257019049.0,-0.022925,SH600507,方大特钢,1.449871e+09,0.0,0.0,SH600507,方大特钢,1.312697e+09
89449,2017-11-06,SH600507,方大特钢,3316,1.326093e+09,1.326093e+09,1.507964e+10,1.670877e+06,11.371481,11.172931,1.941748,3.205201e+07,354122275.0,0.019417,SH600507,方大特钢,1.449871e+09,0.0,0.0,SH600507,方大特钢,1.312697e+09
89450,2017-11-07,SH600507,方大特钢,3317,1.326093e+09,1.326093e+09,1.565410e+10,1.734530e+06,11.804680,11.290256,3.809524,5.189485e+07,609923555.0,0.038095,SH600507,方大特钢,1.449871e+09,0.0,0.0,SH600507,方大特钢,1.312697e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89685,2018-10-25,SH600507,方大特钢,3552,1.449871e+09,1.326093e+09,1.565861e+10,1.432180e+06,10.800000,10.400000,1.313321,2.514361e+07,270726141.0,0.013133,SH600507,方大特钢,1.449871e+09,0.0,0.0,SH600507,方大特钢,1.312697e+09
89686,2018-10-26,SH600507,方大特钢,3553,1.449871e+09,1.326093e+09,1.591959e+10,1.456050e+06,10.980000,10.950000,1.666667,2.048806e+07,226540963.0,0.016667,SH600507,方大特钢,1.449871e+09,0.0,0.0,SH600507,方大特钢,1.312697e+09
89687,2018-10-29,SH600507,方大特钢,3554,1.449871e+09,1.326093e+09,1.593409e+10,1.457376e+06,10.990000,11.100000,0.091075,2.029474e+07,226153364.0,0.000911,SH600507,方大特钢,1.449871e+09,0.0,0.0,SH600507,方大特钢,1.312697e+09
89688,2018-10-30,SH600507,方大特钢,3555,1.449871e+09,1.326093e+09,1.647054e+10,1.506442e+06,11.360000,11.000000,3.366697,2.260068e+07,254053075.0,0.033667,SH600507,方大特钢,1.449871e+09,0.0,0.0,SH600507,方大特钢,1.312697e+09


In [81]:
len(set(la_st1[la_st1["Symbol"].isin(dff2)]["Symbol"].values) & set(set(la_st[la_st["tv_rank"] <= 300]["Symbol"])))

18

In [108]:
sto_ck[sto_ck["Name"] == "小天鹅Ａ"]

,Date,Symbol,Name,ListDays,Totalshares,Marketshares,TotalValue,MarketValue,close,open,ret,volume,amt,returns,证券代码_x,证券简称,A股合计,B股合计,H股合计,证券代码_y,证券名称,净利润


In [109]:
sto_ck = pd.read_csv('F:\\all_stock_201810.csv', encoding = "GBK").iloc[:,1:]
sto_ck[sto_ck["Name"] == "小天鹅Ａ"]

,Date,Symbol,Name,ListDays,Totalshares,Marketshares,TotalValue,MarketValue,close,open,ret,volume,amt
347741,20171101,SZ000418,小天鹅Ａ,4845,632487764.0,438500147.0,3.622890e+06,2.511729e+06,56.381490,57.995765,-2.185792,3.735665e+06,2.102705e+08
347742,20171102,SZ000418,小天鹅Ａ,4846,632487764.0,438500147.0,3.668429e+06,2.543301e+06,57.090196,55.662941,1.256983,3.623374e+06,2.039916e+08
347743,20171103,SZ000418,小天鹅Ａ,4847,632487764.0,438500147.0,3.574821e+06,2.478403e+06,55.633412,56.598039,-2.551724,3.396775e+06,1.925190e+08
347744,20171106,SZ000418,小天鹅Ａ,4848,632487764.0,438500147.0,3.779114e+06,2.620038e+06,58.812745,55.909020,5.714791,4.783298e+06,2.768018e+08
347745,20171107,SZ000418,小天鹅Ａ,4849,632487764.0,438500147.0,3.750652e+06,2.600306e+06,58.369804,58.566667,-0.753138,2.168739e+06,1.263846e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...
347951,20180906,SZ000418,小天鹅Ａ,5055,632487764.0,439364147.0,2.701355e+06,1.876524e+06,42.710000,44.180000,-3.327297,2.683440e+06,1.162424e+08
347952,20180907,SZ000418,小天鹅Ａ,5056,632487764.0,439364147.0,2.941068e+06,2.043043e+06,46.500000,42.930000,8.873800,5.620791e+06,2.560336e+08
347953,20181029,SZ000418,小天鹅Ａ,5057,632487764.0,439364147.0,2.709578e+06,1.882236e+06,42.840000,44.660000,-7.870968,7.737271e+06,3.332459e+08
347954,20181030,SZ000418,小天鹅Ａ,5058,632487764.0,439364147.0,2.719697e+06,1.889266e+06,43.000000,41.520000,0.373483,6.428473e+06,2.716895e+08


In [30]:
sto_ck = pd.read_csv('F:\\all_stock_202004.csv', encoding = "GBK").iloc[:,1:]

sto_ck[sto_ck["Symbol"].isin(list(set(set(total_list) - set(new_list)) - set(la_st1["Symbol"])))].groupby("Symbol")["Name"].first()

Symbol
SZ000536    华映科技
SZ000727    华东科技
SZ002176    江特电机
SZ002426    胜利精密
Name: Name, dtype: object

In [20]:
len(dff2)

50

In [106]:
la_st1[la_st1["Symbol"].isin(list(set(old_list) - set(list2)))].loc[:,
       ["Symbol", "Name", "daily_tv", "daily_amt"]]

,Symbol,Name,daily_tv,daily_amt
412,SH600549,厦门钨业,1.934945e+10,1.943928e+08
567,SH600739,辽宁成大,1.902908e+10,2.618176e+08
705,SH600909,华安证券,2.121068e+10,2.770312e+08
747,SH600998,九州通,2.858037e+10,6.549084e+07
829,SH601333,广深铁路,1.953899e+10,1.270550e+08
859,SH601611,中国核建,2.067871e+10,1.542895e+08
928,SH601991,大唐发电,4.052189e+10,5.414535e+07
1470,SZ000408,藏格控股,2.306243e+10,6.519863e+07
1490,SZ000503,国新健康,1.672983e+10,3.876022e+08
1712,SZ000826,启迪桑德,1.691110e+10,2.368684e+08


In [89]:
import numpy as np
sto_ck = pd.read_csv('F:\\all_stock_201910.csv', encoding = "GBK").iloc[:,1:]
st_list = sto_ck[(sto_ck["Name"].str.contains("ST") != False)  |
             (sto_ck["Name"].str.contains("退") != False)]["Symbol"].unique()
sto_ck = sto_ck[~sto_ck["Symbol"].isin(st_list)]
# sto_ck.loc[sto_ck["Symbol"] == "SZ000022", "Symbol"] = "SZ001872"
# sto_ck[sto_ck["Symbol"] == "SZ001872"]
# sto_ck.loc[sto_ck["Symbol"] == "SZ000043", "Symbol"] = "SZ001914"
# sto_ck[sto_ck["Symbol"] == "SZ001914"]

ts = pd.read_csv('F:\\total shares 201910.csv')
ts = ts.rename(columns={ts.columns[2]:"A股合计"})
ts = ts.rename(columns={ts.columns[3]:"B股合计"})
ts = ts.rename(columns={ts.columns[4]:"H股合计"})
ts["A股合计"] = ts["A股合计"].astype(str)
ts["B股合计"] = ts["B股合计"].astype(str)
ts["H股合计"] = ts["H股合计"].astype(str)
ts["A股合计"] = ts["A股合计"].apply(lambda x: float(x.replace(',', "")))
ts["B股合计"] = ts["B股合计"].apply(lambda x: float(x.replace(',', "")))
ts["H股合计"] = ts["H股合计"].apply(lambda x: float(x.replace(',', "")))
ts["证券代码"] = ts["证券代码"].str[-2:] + ts["证券代码"].str[:-3] 
sto_ck = pd.merge(sto_ck, ts, left_on="Symbol", right_on="证券代码", how="left")
display(sto_ck[sto_ck["A股合计"].isnull()]["Name"].unique())
sto_ck = sto_ck[~sto_ck["A股合计"].isnull()]

sto_ck["Totalshares"] = np.where((sto_ck["B股合计"] != 0)|(sto_ck["H股合计"] != 0), sto_ck["A股合计"], sto_ck["Totalshares"])
sto_ck["TotalValue"] = sto_ck["Totalshares"] * sto_ck["close"]

ir = pd.read_csv('F:\\2019中报净利润.csv', encoding = "GBK")
ir = ir.rename(columns={ir.columns[2]:"净利润"})
ir["净利润"] = np.where(ir["净利润"] == '——', 0, ir["净利润"])
ir["净利润"] = ir["净利润"].astype(str)
ir["净利润"] = ir["净利润"].apply(lambda x: float(x.replace(',', "")))
ir["证券代码"] = ir["证券代码"].str[-2:] + ir["证券代码"].str[:-3] 
sto_ck = pd.merge(sto_ck, ir, left_on="Symbol", right_on="证券代码", how="left")
display(sto_ck[sto_ck["净利润"].isnull()]["Name"].unique())

array(['外运发展', '深赤湾Ａ', '中航善达', '小天鹅Ａ'], dtype=object)

array([], dtype=object)